<a href="https://colab.research.google.com/github/muhammetsnts/SPARK/blob/main/projects/9.Spam_Detection_NLP_with_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [1]:
# install Java8
!apt-get -q install openjdk-8-jdk-headless -qq > /dev/null

# download spark3.1.1
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

# unzip it
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

# install findspark 
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark = SparkSession.builder.appName('ops').getOrCreate()

# Download And Read Dataset

In [2]:
!wget -q https://raw.githubusercontent.com/muhammetsnts/SPARK/main/data/SMSSpamCollection

In [5]:
data = spark.read.csv("SMSSpamCollection", inferSchema=True, sep="\t")
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length', length(data['text']))

data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



We can see that there is a clear difference between the length of ham and spam messages.

# Create NLP Pipeline

In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [12]:
tokenizer = Tokenizer(inputCol='text', outputCol='token')
stop_remove = StopWordsRemover(inputCol='token', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [17]:
from pyspark.ml import Pipeline

In [19]:
pipeline = Pipeline(stages=[
                           ham_spam_to_numeric,
                           tokenizer,
                           stop_remove,
                           count_vec,
                           idf,
                           clean_up    
])

In [21]:
cleaner = pipeline.fit(data).transform(data)

In [23]:
cleaner.columns

['class',
 'text',
 'length',
 'label',
 'token',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [24]:
cleaner = cleaner.select('label', 'features')

cleaner.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



# Train-Test Split

In [25]:
training, test = cleaner.randomSplit([0.7, 0.3])

# Create Model
We will create a NaiveBayes model for classification.

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes()

In [26]:
spam_detector = nb.fit(training)

In [27]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [28]:
result = spam_detector.transform(test)

In [29]:
result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1058.9167940380...|[1.0,5.5985807605...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-616.04038306338...|[0.99960011990334...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1138.7317512135...|[6.18289713080342...|       1.0|
|  0.0|(13424,[0,1,498,5...|[-322.68679960841...|[0.99999999996447...|       0.0|
|  0.0|(13424,[0,1,3657,...|[-128.14775659002...|[0.99997392865105...|       0.0|
|  0.0|(13424,[0,2,3,5,3...|[-491.01721738697...|[1.0,1.8542919091...|       0.0|
|  0.0|(13424,[0,2,4,7,2...|[-519.20910873634...|[1.0,4.4988356840...|       0.0|
|  0.0|(13424,[0,2,4,25,...|[-440.01404086094...|[1.0,6.2376563134...|       0.0|
|  0.0|(13424,[0,2,4,40,...|[-1582.8569850404...|[0.99998975317248...|       0.0|
|  0.0|(13424,[0

# Evaluation

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval = MulticlassClassificationEvaluator()

In [32]:
acc = acc_eval.evaluate(result)

In [33]:
print(f'Acc of NB Model: ',acc)

Acc of NB Model:  0.9242987690816107
